# Requirements

In [ ]:
# Add as many imports as you need.

In [12]:
import pandas as pd

In [1]:
# Add as many imports as you need.
# Write your code here. Add as many boxes as you need.

# Add as many imports as you need.
# Add as many imports as you need.
import pandas as pd

from VNP.packages.data_alalysis import *
from VNP.packages.data_preperation import *
from VNP.packages.missing_values import *
from VNP.packages.visualization import *
from VNP.packages.models import *
from VNP.packages.evaluation import *
from VNP.packages.vnp_ez import *
%load_ext autoreload
%autoreload 2

In [2]:
from keras.api.models import Sequential
from keras.api.layers import Input, LSTM, Dense
import keras

# Laboratory Exercise - Run Mode (8 points)

## Introduction
In this laboratory assignment, the primary objective is to use Long Short-Term Memory (LSTM) networks for time series forecasting in order to predict the current **close price** of the Dow Jones Industrial Average index. To accomplish this use data from the past 7 days, which includes numeric information and news information. The goal is to employ LSTM, a type of recurrent neural network, to effectively forecast one future step for the index price (the following day).


## The DIJA Dataset

This dataset consists of daily price records for the value of the Dow Jones Industrial Average index. The dataset includes the following attributes:

- Date - date in the format YYYY-MM-DD,
- Open - open price of the index on the specified date
- Close - close price of the index on the specified date
- High - high price of the index on the specified date
- Low - low price of the index on the specified date
- Volume - number of trades



## The Reddit News Dataset

This dataset consists of news headlines for a certain date that might impact the price:

- Date - date in the format YYYY-MM-DD,
- News - news headline scraped from Reddit

<b>Note: You might have multiple headlines for each date. The number of news per date might not be the same for each date. <b>

Load the datasets into a `pandas` data frame.

In [6]:
# Write your code here. Add as many boxes as you need.
df_1 = pd.read_csv("../data/DJIA_table.csv")

In [7]:
df_1

,Date,Open,High,Low,Close,Volume
0,7/1/2016,17924.24023,18002.38086,17916.91016,17949.36914,82160000
1,6/30/2016,17712.75977,17930.60938,17711.80078,17929.99023,133030000
2,6/29/2016,17456.01953,17704.50977,17456.01953,17694.67969,106380000
3,6/28/2016,17190.50977,17409.72070,17190.50977,17409.72070,112190000
4,6/27/2016,17355.21094,17355.21094,17063.08008,17140.24023,138740000
...,...,...,...,...,...,...
1984,8/14/2008,11532.07031,11718.28027,11450.88965,11615.92969,159790000
1985,8/13/2008,11632.80957,11633.78027,11453.33984,11532.95996,182550000
1986,8/12/2008,11781.70020,11782.34961,11601.51953,11642.46973,173590000
1987,8/11/2008,11729.66992,11867.11035,11675.53027,11782.34961,183190000


In [8]:
df_2 = pd.read_csv("../data/RedditNews.csv")

In [9]:
df_2

,Date,News
0,2016-07-01,A 117-year-old woman in Mexico City finally re...
1,2016-07-01,IMF chief backs Athens as permanent Olympic host
2,2016-07-01,"The president of France says if Brexit won, so..."
3,2016-07-01,British Man Who Must Give Police 24 Hours' Not...
4,2016-07-01,100+ Nobel laureates urge Greenpeace to stop o...
...,...,...
73603,2008-06-08,b'Man goes berzerk in Akihabara and stabs ever...
73604,2008-06-08,b'Threat of world AIDS pandemic among heterose...
73605,2008-06-08,b'Angst in Ankara: Turkey Steers into a Danger...
73606,2008-06-08,"b""UK: Identity cards 'could be used to spy on ..."


In [10]:
df_1_sorted = to_time_series(df_1, time_feature="Date", auto_sort=True)

In [11]:
df_2_sorted = to_time_series(df_2, time_feature="Date", auto_sort=True)

In [12]:
df_1_sorted

,Open,High,Low,Close,Volume
Date,,,,,
2008-08-08,11432.08984,11759.95996,11388.04004,11734.32031,212830000
2008-08-11,11729.66992,11867.11035,11675.53027,11782.34961,183190000
2008-08-12,11781.70020,11782.34961,11601.51953,11642.46973,173590000
2008-08-13,11632.80957,11633.78027,11453.33984,11532.95996,182550000
2008-08-14,11532.07031,11718.28027,11450.88965,11615.92969,159790000
...,...,...,...,...,...
2016-06-27,17355.21094,17355.21094,17063.08008,17140.24023,138740000
2016-06-28,17190.50977,17409.72070,17190.50977,17409.72070,112190000
2016-06-29,17456.01953,17704.50977,17456.01953,17694.67969,106380000


In [13]:
df_2_sorted.groupby(pd.Grouper(freq="1D")).first()

,News
Date,
2008-06-08,"b'Marriage, they said, was reduced to the stat..."
2008-06-09,"b'Chew Qat: In Yemen, 72 per cent of men and 3..."
2008-06-10,b'31 year old beats 3 year old to death: worst...
2008-06-11,"b""Pakistan Blames U.S. Coalition for Troops' D..."
2008-06-12,"b'Marine ""Mean Motari"" gets expelled, another ..."
...,...
2016-06-27,Angela Merkel said the U.K. must file exit pap...
2016-06-28,Hong Kong democracy activists call for return ...
2016-06-29,A chatbot programmed by a British teenager has...


In [14]:
df = pd.merge(df_1_sorted, df_2_sorted, left_index=True, right_index=True ,how="left")

In [15]:
df_3 = df.groupby(pd.Grouper(freq="1D")).first()

In [16]:
df_3.isna().sum()

Open      896
High      896
Low       896
Close     896
Volume    896
News      896
dtype: int64

In [17]:
df_4 = df_3.dropna()

In [18]:
df_4

,Open,High,Low,Close,Volume,News
Date,,,,,,
2008-08-08,11432.08984,11759.95996,11388.04004,11734.32031,212830000.0,b'Georgian troops retreat from S. Osettain cap...
2008-08-11,11729.66992,11867.11035,11675.53027,11782.34961,183190000.0,b'Russia angered by Israeli military sale to G...
2008-08-12,11781.70020,11782.34961,11601.51953,11642.46973,173590000.0,b'U.S. Beats War Drum as Iran Dumps the Dollar'
2008-08-13,11632.80957,11633.78027,11453.33984,11532.95996,182550000.0,"b""Bush announces Operation Get All Up In Russi..."
2008-08-14,11532.07031,11718.28027,11450.88965,11615.92969,159790000.0,b'Poland and US agree to missle defense deal. ...
...,...,...,...,...,...,...
2016-06-27,17355.21094,17355.21094,17063.08008,17140.24023,138740000.0,Angela Merkel said the U.K. must file exit pap...
2016-06-28,17190.50977,17409.72070,17190.50977,17409.72070,112190000.0,Hong Kong democracy activists call for return ...
2016-06-29,17456.01953,17704.50977,17456.01953,17694.67969,106380000.0,A chatbot programmed by a British teenager has...


In [19]:
df_4.isna().sum()

Open      0
High      0
Low       0
Close     0
Volume    0
News      0
dtype: int64

In [20]:
from transformers import AutoTokenizer

checkpoint = "google-bert/bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [21]:
def tokenize(sample):
    return tokenizer(sample["News"], truncation=True)

In [22]:
tokenized_dataset = tokenizer(df_4["News"].tolist(), truncation=True)

In [24]:
df_5 = df_4.copy()

In [28]:
token_ids = pd.DataFrame({"token_ids":tokenized_dataset["input_ids"]}, index=df_5.index)

In [30]:
df_6 = pd.concat([df_5, token_ids], axis=1)

In [31]:
df_6

,Open,High,Low,Close,Volume,News,token_ids
Date,,,,,,,
2008-08-08,11432.08984,11759.95996,11388.04004,11734.32031,212830000.0,b'Georgian troops retreat from S. Osettain cap...,"[101, 170, 112, 49857, 20836, 75559, 10188, 15..."
2008-08-11,11729.66992,11867.11035,11675.53027,11782.34961,183190000.0,b'Russia angered by Israeli military sale to G...,"[101, 170, 112, 14664, 46464, 10336, 10155, 28..."
2008-08-12,11781.70020,11782.34961,11601.51953,11642.46973,173590000.0,b'U.S. Beats War Drum as Iran Dumps the Dollar',"[101, 170, 112, 158, 119, 156, 119, 97453, 112..."
2008-08-13,11632.80957,11633.78027,11453.33984,11532.95996,182550000.0,"b""Bush announces Operation Get All Up In Russi...","[101, 170, 107, 21145, 11671, 79603, 21188, 17..."
2008-08-14,11532.07031,11718.28027,11450.88965,11615.92969,159790000.0,b'Poland and US agree to missle defense deal. ...,"[101, 170, 112, 17879, 10111, 10808, 68312, 10..."
...,...,...,...,...,...,...,...
2016-06-27,17355.21094,17355.21094,17063.08008,17140.24023,138740000.0,Angela Merkel said the U.K. must file exit pap...,"[101, 27365, 100424, 12415, 10105, 158, 119, 1..."
2016-06-28,17190.50977,17409.72070,17190.50977,17409.72070,112190000.0,Hong Kong democracy activists call for return ...,"[101, 15078, 14577, 67519, 84024, 20575, 10142..."
2016-06-29,17456.01953,17704.50977,17456.01953,17694.67969,106380000.0,A chatbot programmed by a British teenager has...,"[101, 138, 69338, 25022, 19611, 10162, 10155, ..."


Merge the datasets (be careful you can get multiple rows per date which is not desirable)

In [ ]:
# Write your code here. Add as many boxes as you need.

## Feauture Extraction


1. DIJA Table
Apply a lag of one, up to 7 days to each feature, creating a set of features representing the index price from the previous 7 days. To maintain dataset integrity, eliminate any resulting missing values at the beginning of the dataset.

2. Reddit News Table
Create a numeric representation for the news (for example average embedding or average sentiment). <b> You must create lags of the news features as well since we will not know the news for the future. </b>

Hint: Use `df['column_name'].shift(period)`. Check the documentation at https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shift.html.

In [ ]:
# Write your code here. Add as many boxes as you need.

## Dataset Splitting
Partition the dataset into training and testing sets with an 80:20 ratio.

**WARNING: DO NOT SHUFFLE THE DATASET.**



In [ ]:
# Write your code here. Add as many boxes as you need.

## Feauture Scaling
Scale the extracted features using an appropriate scaler if needed.

In [ ]:
# Write your code here. Add as many boxes as you need.

## Feature Reshaping

Reshape the feature dimensions into the shape `(samples, timesteps, features)`.

In [ ]:
# Write your code here. Add as many boxes as you need.

## Long Short-Term Memory (LSTM) Network


Define the forecasting model using the **Keras Sequential API** (`keras.models.Sequential`), incorporating one or more LSTM layers along with additional relevant layers (`keras.layers`). Be cautious when specifying the configuration of the final layer to ensure proper model output for the forecasting task.

In [ ]:
# Write your code here. Add as many boxes as you need.

Compile the previously defined model specifying **loss function** (`keras.losses`), **optimizer** (`keras.optimizers`) and **evaluation metrics** (`keras.metics`).

In [ ]:
# Write your code here. Add as many boxes as you need.

Train the model on the training set, specifying the **batch size** and **number of epochs** for the training process. Allocate 20% of the samples for **validation**, and ensure that the dataset remains **unshuffled** during training.

In [ ]:
# Write your code here. Add as many boxes as you need.

Create a line plot illustrating both the **training** and **validation loss** over the training epochs.

In [ ]:
# Write your code here. Add as many boxes as you need.

Use the trained model to make predictions for the test set.

In [ ]:
# Write your code here. Add as many boxes as you need.

Assess the performance of the model by using different metrics provided by the `scikit-learn` library.

In [ ]:
# Write your code here. Add as many boxes as you need.

Create a line plot in order to compare the actual and predicted mean temperatures for the test set.

In [ ]:
# Write your code here. Add as many boxes as you need.